# Multi-VAE

이번 미션에서는 [Variational Autoencoders for Collaborative Filtering](https://arxiv.org/abs/1802.05814)에서 제안된 Multi-VAE 기반의 협업 필터링을 구현해보도록 하겠습니다. 다양한 Auto-Encoder 기반의 협업필터링이 제안된 이후에, 가장 강력하다고 평가받는 VAE 기반의 협업 필터링을 이해하는 시간을 갖도록 하겠습니다.

- 이 미션은 다음 [코드](https://github.com/younggyoseo/vae-cf-pytorch)를 기반으로 작성되었습니다. 바로 코드를 확인해보지 마시고, 최대한 직접 작성을 해보세요!
- 이 미션에서 중요한 부분은 모델 부분입니다. 데이터 전처리 부분은 가볍게 훑어 보시고, 모델 부분을 집중해주세요!
- 완성을 해야할 부분은 TODO로 표시가 되어있습니다.

## 1. 초기 세팅

In [ ]:
## 전처리과정에서 pandas의 버전에 다르게 동작하는 경향이 보여, 이 미션에서는 아래 버전으로 사용하도록하겠습니다.
# !pip install pandas==1.0.1

In [ ]:
import argparse
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from scipy import sparse


### 데이터 다운로드
이곳에 대회 사이트(AI Stages)에 있는 data의 URL을 입력해주세요. 
- 데이터 URL은 변경될 수 있습니다.
- 예) `!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000176/data/data.tar.gz`

In [ ]:
# !wget <대회 데이터 URL>
# !tar -xf data.tar.gz

In [ ]:
## 각종 파라미터 세팅
parser = argparse.ArgumentParser(description='PyTorch Variational Autoencoders for Collaborative Filtering')


parser.add_argument('--data', type=str, default='',
                    help='Movielens dataset location')

parser.add_argument('--lr', type=float, default=1e-4,
                    help='initial learning rate')
parser.add_argument('--wd', type=float, default=0.00,
                    help='weight decay coefficient')
parser.add_argument('--batch_size', type=int, default=500,
                    help='batch size')
parser.add_argument('--epochs', type=int, default=20,
                    help='upper epoch limit')
parser.add_argument('--total_anneal_steps', type=int, default=200000,
                    help='the total number of gradient updates for annealing')
parser.add_argument('--anneal_cap', type=float, default=0.2,
                    help='largest annealing parameter')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--cuda', action='store_true',
                    help='use CUDA')
parser.add_argument('--log_interval', type=int, default=100, metavar='N',
                    help='report interval')
parser.add_argument('--save', type=str, default='model.pt',
                    help='path to save the final model')
args = parser.parse_args([])

# Set the random seed manually for reproductibility.
torch.manual_seed(args.seed)

#만약 GPU가 사용가능한 환경이라면 GPU를 사용
if torch.cuda.is_available():
    args.cuda = True

device = torch.device("cuda" if args.cuda else "cpu")
device

## 2. 데이터 전처리

이 부분에서 진행되는 과정은 저희가 일반적으로 알고있는 MovieLens (user, item, timestamp)데이터를 전처리하는 과정입니다. 전처리 과정의 다양한 옵션들을 구성하기 위해 약간 복잡하게 되었지만, 
결과적으로는, 유저들의 특정한 아이템들을 따로 분리를 해서, 그 분리된 값을 모델이 예측할 수 있냐를 확인하기 위한 전처리 과정이라고 보시면 되겠습니다.
실제로 나오는 데이터셋을 확인하면 더욱 이해가 빠를것입니다.

In [ ]:
import os
import pandas as pd
from scipy import sparse
import numpy as np

# 데이터 tp 의 항목 id 가 몇개인지 세주는 함수
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()

    return count

# 특정한 횟수 이상의 리뷰가 존재하는(사용자의 경우 min_uc 이상, 아이템의 경우 min_sc이상) 
# 데이터만을 추출할 때 사용하는 함수입니다.
# 현재 데이터셋에서는 결과적으로 원본그대로 사용하게 됩니다.

# 최소 유저, 아이템 이상의 데이터들만 뽑아서 데이터 변환. count 도 포함해서 return
def filter_triplets(tp, min_uc=5, min_sc=0):
    if min_sc > 0:
        itemcount = get_count(tp, 'item')
        tp = tp[tp['item'].isin(itemcount.index[itemcount >= min_sc])]

    if min_uc > 0:
        usercount = get_count(tp, 'user')
        tp = tp[tp['user'].isin(usercount.index[usercount >= min_uc])]

    usercount, itemcount = get_count(tp, 'user'), get_count(tp, 'item')
    return tp, usercount, itemcount

#훈련된 모델을 이용해 검증할 데이터를 분리하는 함수입니다.
#100개의 액션이 있다면, 그중에 test_prop 비율 만큼을 비워두고, 그것을 모델이 예측할 수 있는지를
#확인하기 위함입니다.
def split_train_test_proportion(data, test_prop=0.2):
    #데이터를 유저별로 묶고
    data_grouped_by_user = data.groupby('user')
    tr_list, te_list = list(), list()

    np.random.seed(98765)
    
    for _, group in data_grouped_by_user:
        n_items_u = len(group)
        
        #평가 5개 이상인 것들은 
        if n_items_u >= 5:
            # 전체 False 인 numpy 생성
            idx = np.zeros(n_items_u, dtype='bool')
            #test_prop 비율 만큼 랜덤으로 True 값으로 변경
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        
        #평가 5개이하 한것들은 train으로
        else:
            tr_list.append(group)
    
    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)

    return data_tr, data_te

def numerize(tp, profile2id, show2id):
    uid = tp['user'].apply(lambda x: profile2id[x])
    sid = tp['item'].apply(lambda x: show2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [ ]:
print("Load and Preprocess Movielens dataset")
# Load Data
DATA_DIR = args.data
raw_data = pd.read_csv(os.path.join(DATA_DIR, 'useritem.csv'), header=0)
print("원본 데이터\n", raw_data)

# Filter Data
raw_data, user_activity, item_popularity = filter_triplets(raw_data, min_uc=1, min_sc=0)
#제공된 훈련데이터의 유저는 모두 5개 이상의 리뷰가 있습니다.
print("5번 이상의 리뷰가 있는 유저들로만 구성된 데이터\n",raw_data)

print("유저별 리뷰수\n",user_activity)
print("아이템별 리뷰수\n",item_popularity)

In [ ]:
# Shuffle User Indices
unique_uid = user_activity.index
print("(BEFORE) unique_uid:",unique_uid)
np.random.seed(98765)
idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]
print("(AFTER) unique_uid:",unique_uid)

n_users = unique_uid.size #31360
n_heldout_users = 3000


# Split Train/Validation/Test User Indices
tr_users = unique_uid[:(n_users - n_heldout_users * 2)]
vd_users = unique_uid[(n_users - n_heldout_users * 2): (n_users - n_heldout_users)]
te_users = unique_uid[(n_users - n_heldout_users):]
sub_users = unique_uid
#주의: 데이터의 수가 아닌 사용자의 수입니다!
print("훈련 데이터에 사용될 사용자 수:", len(tr_users))
print("검증 데이터에 사용될 사용자 수:", len(vd_users))
print("테스트 데이터에 사용될 사용자 수:", len(te_users))
print("제출 데이터에 사용될 사용자 수:", len(sub_users))

In [ ]:
#훈련 데이터에 해당하는 아이템들
# Train에는 전체 데이터를 사용합니다.
train_plays = raw_data.loc[raw_data['user'].isin(tr_users)]

sub_plays = raw_data.loc[raw_data['user'].isin(sub_users)]

#아이템 ID
unique_sid = pd.unique(train_plays['item'])
unique_sub_sid = pd.unique(sub_plays['item'])

id2user = dict((i, pid) for (i, pid) in enumerate(pd.unique(sub_plays['user'])))
id2item = dict((i, pid) for (i, pid) in enumerate(pd.unique(sub_plays['item'])))

show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

sub_show2id = dict((sid, i) for (i, sid) in enumerate(unique_sub_sid))
sub_profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

pro_dir = os.path.join(DATA_DIR, 'pro_sg')

if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)

with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

with open(os.path.join(pro_dir, 'unique_sub_sid.txt'), 'w') as f:
    for sid in unique_sub_sid:
        f.write('%s\n' % sid)

# Validation과 Test에는 input으로 사용될 tr 데이터와 정답을 확인하기 위한 te 데이터로 분리되었습니다.
vad_plays = raw_data.loc[raw_data['user'].isin(vd_users)]
vad_plays = vad_plays.loc[vad_plays['item'].isin(unique_sid)]
vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

test_plays = raw_data.loc[raw_data['user'].isin(te_users)]
test_plays = test_plays.loc[test_plays['item'].isin(unique_sid)]
test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)



train_data = numerize(train_plays, profile2id, show2id)
train_data.to_csv(os.path.join(pro_dir, 'train.csv'), index=False)

sub_data = numerize(sub_plays, sub_profile2id, sub_show2id)
sub_data.to_csv(os.path.join(pro_dir, 'sub.csv'), index=False)


vad_data_tr = numerize(vad_plays_tr, profile2id, show2id)
vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr.csv'), index=False)

vad_data_te = numerize(vad_plays_te, profile2id, show2id)
vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te.csv'), index=False)

test_data_tr = numerize(test_plays_tr, profile2id, show2id)
test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr.csv'), index=False)

test_data_te = numerize(test_plays_te, profile2id, show2id)
test_data_te.to_csv(os.path.join(pro_dir, 'test_te.csv'), index=False)

print("Done!")

In [ ]:
#데이터 셋 확인
print(train_data)
print(vad_data_tr)
print(vad_data_te)
# print(test_data_tr)
# print(test_data_te)

In [ ]:

path = os.path.join(f'pro_sg/sub.csv')

tp = pd.read_csv(path)
n_users = tp['uid'].max() + 1
n_items = tp['sid'].max() + 1
rows, cols = tp['uid'], tp['sid']
#compressed sparse row matrix로 변환하기 (희소행렬을 다른식으로 변환하여 저장하는 방법)
data = sparse.csr_matrix((np.ones_like(rows),
                            (rows, cols)), dtype='float64',
                            shape=(n_users, n_items))

## 3. 데이터 로더 설정

In [ ]:

class DataLoader():
    '''
    Load Movielens dataset
    '''
    def __init__(self, path):
        
        self.pro_dir = os.path.join(path, 'pro_sg')
        #error 설정
        assert os.path.exists(self.pro_dir), "Preprocessed files do not exist. Run data.py"
        # load_n_items 를 통해 이전에 저장해뒀던 아이템의 랜덤 순서 불러옴
        self.n_items = self.load_n_items()
    
    def load_data(self, datatype='train'):
        if datatype == 'train':
            return self._load_train_data()
        elif datatype == 'validation':
            return self._load_tr_te_data(datatype)
        elif datatype == 'test':
            return self._load_tr_te_data(datatype)
        elif datatype == 'sub':
            return self._load_train_data(datatype)
        else:
            raise ValueError("datatype should be in [train, validation, test, submission]")
    
    # self.n_items 에 이전에 저장해뒀던 아이템의 랜덤순서 가져다주는 함수
    def load_n_items(self):
        unique_sid = list()
        with open(os.path.join(self.pro_dir, 'unique_sub_sid.txt'), 'r') as f:
            for line in f:
                unique_sid.append(line.strip())
        n_items = len(unique_sid)
        return n_items
    
    def _load_train_data(self, datatype = 'train'):
        path = os.path.join(self.pro_dir, f'{datatype}.csv')
        
        tp = pd.read_csv(path)
        n_users = tp['uid'].max() + 1

        rows, cols = tp['uid'], tp['sid']
        #compressed sparse row matrix로 변환하기 (희소행렬을 다른식으로 변환하여 저장하는 방법)
        data = sparse.csr_matrix((np.ones_like(rows),
                                 (rows, cols)), dtype='float64',
                                 shape=(n_users, self.n_items))
        return data
    
    def _load_tr_te_data(self, datatype='test'):
        tr_path = os.path.join(self.pro_dir, '{}_tr.csv'.format(datatype))
        te_path = os.path.join(self.pro_dir, '{}_te.csv'.format(datatype))

        tp_tr = pd.read_csv(tr_path)
        tp_te = pd.read_csv(te_path)

        start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
        end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

        rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
        rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

        data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                                    (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, self.n_items))
        data_te = sparse.csr_matrix((np.ones_like(rows_te),
                                    (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, self.n_items))
        return data_tr, data_te

## 4. 모델정의

VAE 코드 참고: https://atcold.github.io/pytorch-Deep-Learning/ko/week08/08-3/

multi-vae 는 multinomial likelihood를 사용하기 때문에 implicit feedback data를 더 잘 설명할 수 있다고 한다.

Multi-vae 이론 참고: https://velog.io/@2712qwer/Paper-Code-Review-2018-WWW-Variational-Autoencoders-for-Collaborative-Filtering

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np


#이미 완성된 MultiDAE(denoising auto encoder)의 코드를 참고하여 그 아래 MultiVAE의 코드를 완성해보세요!
class MultiDAE(nn.Module):
    """
    Container module for Multi-DAE.

    Multi-DAE : Denoising Autoencoder with Multinomial Likelihood
    See Variational Autoencoders for Collaborative Filtering
    https://arxiv.org/abs/1802.05814
    """

    def __init__(self, p_dims, q_dims=None, dropout=0.5):
        super(MultiDAE, self).__init__()
        #p_dims, q_dims 는 input, output dimension 리스트
        #p_dims = [200, 600, 6807]
        self.p_dims = p_dims
        #q_dims
        if q_dims:
            assert q_dims[0] == p_dims[-1], "In and Out dimensions must equal to each other"
            assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q- network mismatches."
            self.q_dims = q_dims
        # q_dims 없으면 p_dims 순서 뒤집어서 사용
        else:
            self.q_dims = p_dims[::-1]
        # 항목이 5개가 되게 함
        self.dims = self.q_dims + self.p_dims[1:]
        # nn.Sequential 과 비슷한함수로, Module 여러개 담아놓는 역할
        # nn.Linear(6807, 600), nn.Linear(600,200),nn.Linear(200, 600),nn.Linear(600,6807)
        self.layers = nn.ModuleList([nn.Linear(d_in, d_out) for
            d_in, d_out in zip(self.dims[:-1], self.dims[1:])])
        self.drop = nn.Dropout(dropout)
        
        self.init_weights()
    
    def forward(self, input):
        #input 정규화 (이유 : 학습 속도 높이고, Local optimum에 빠지게 하지 않기 위해)
        #input dropout 으로 몇가닥 끊기(과적합 방지)
        h = F.normalize(input)
        h = self.drop(h)
        #nn.Module에 저장해 뒀던 Linear 함수 적용
        for i, layer in enumerate(self.layers):
            h = layer(h)
            # 마지막 항에서는 tanh로 activation function 적용
            if i != len(self.layers) - 1:
                h = F.tanh(h)
        return h

    def init_weights(self):
        # 가중치 초기화 하는 함수 
        for layer in self.layers:
            # Xavier Initialization for weights
            size = layer.weight.size()
            fan_out = size[0]
            fan_in = size[1]
            std = np.sqrt(2.0/(fan_in + fan_out))
            # 가중치 함수 초기화 (평균=0 , 표준편차 = std)
            layer.weight.data.normal_(0.0, std)

            # Normal Initialization for Biases
            layer.bias.data.normal_(0.0, 0.001)


def loss_function_dae(recon_x, x):
    BCE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
    return BCE



# TODO
# 다양한 VAE의 코드를 다음 코드를 확인한 뒤에, 아래코드에 맞춰서 직접 작성해보는 연습을 해보세요!
# https://github.com/AntixK/PyTorch-VAE
class MultiVAE(nn.Module):
    """
    Container module for Multi-VAE.

    Multi-VAE : Variational Autoencoder with Multinomial Likelihood
    See Variational Autoencoders for Collaborative Filtering
    https://arxiv.org/abs/1802.05814
    """

    def __init__(self, p_dims, q_dims=None, dropout=0.5):
        super(MultiVAE, self).__init__()
        # init 부분은 Multi DAE 와 동일
        self.p_dims = p_dims
        if q_dims:
            assert q_dims[0] == p_dims[-1], "In and Out dimensions must equal to each other"
            assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q- network mismatches."
            self.q_dims = q_dims
        else:
            self.q_dims = p_dims[::-1]

        # Last dimension of q- network is for mean and variance
        temp_q_dims = self.q_dims[:-1] + [self.q_dims[-1] * 2]
        # encoder 용 : q_layers는 p_dims 뒤집고 마지막항 한번더 연산추가한 Linear layer 들의 결합
        self.q_layers = nn.ModuleList([nn.Linear(d_in, d_out) for
            d_in, d_out in zip(temp_q_dims[:-1], temp_q_dims[1:])])
        # decoder 용 : p_layer는 p_dims 그대로 사용한 Linear layer 들의 결합으로
        self.p_layers = nn.ModuleList([nn.Linear(d_in, d_out) for
            d_in, d_out in zip(self.p_dims[:-1], self.p_dims[1:])])
        
        self.drop = nn.Dropout(dropout)
        self.init_weights()
    
    # 인풋 -> 인코더 ->  파라미터 재정비 -> 디코더 -> 아웃풋 + 인코더 결과물(mu, logvar)
    def forward(self, input):
        mu, logvar = self.encode(input)
        z = self.reparameterize(mu, logvar)
        h = self.decode(z)
        return h, mu, logvar
    
    def encode(self, input):
        h = F.normalize(input)
        h = self.drop(h)
        #인코더에서는 MultiDAE 처럼 linear layer 돌림
        for i, layer in enumerate(self.q_layers):
            h = layer(h)
            if i != len(self.q_layers) - 1:
                h = F.tanh(h)
            else:
                # mu : 평균
                # logvar : log 분산 (표준편차가 음수가 되지 않기 위한 연산)
                # 처음 값들은 평균(mu)로 보내고 나머지 값들은 분산으로 보냄
                # h 는 [항목수, linear 로 변환된 dim]
                mu = h[:, :self.q_dims[-1]]
                logvar = h[:, self.q_dims[-1]:]
                # 이후 reparameterize 에서 연산처리함
        return mu, logvar

    # training 과정에서 역전파를 수행할 수 있도록 재매개변수화 함수를 따로 생성했다고 함.
    def reparameterize(self, mu, logvar):
        # 학습중일 때는 평균 중심으로 분산 흩뿌려서 제출
        if self.training:
            #logvar로 표준편차 계산
            std = torch.exp(0.5 * logvar)
            # std를 정규분포 값으로 초기화한 eps
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        # 아닐 때는 그냥 평균값 배출
        else:
            return mu

    def decode(self, z):
        h = z
        for i, layer in enumerate(self.p_layers):
            h = layer(h)
            if i != len(self.p_layers) - 1:
                h = F.tanh(h)
        return h

    def init_weights(self):
        for layer in self.q_layers:
            # Xavier Initialization for weights
            size = layer.weight.size()
            fan_out = size[0]
            fan_in = size[1]
            std = np.sqrt(2.0/(fan_in + fan_out))
            layer.weight.data.normal_(0.0, std)

            # Normal Initialization for Biases
            layer.bias.data.normal_(0.0, 0.001)
        
        for layer in self.p_layers:
            # Xavier Initialization for weights
            size = layer.weight.size()
            fan_out = size[0]
            fan_in = size[1]
            std = np.sqrt(2.0/(fan_in + fan_out))
            layer.weight.data.normal_(0.0, std)

            # Normal Initialization for Biases
            layer.bias.data.normal_(0.0, 0.001)



def loss_function_vae(recon_x, x, mu, logvar, anneal=1.0):
    # Loss function은 BCE와 KLD 사용
    # KL annealing 을 통해 Regularization 부여
    BCE = -torch.mean(torch.sum(F.log_softmax(recon_x, 1) * x, -1))
    KLD = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))
    # anneal 값을 0에서부터 특정 값까지 선형적으로 증가시켜 
    # 학습 초기에 reconstruction term을 강조하여 보다 효율적인 학습 도모함.
    return BCE + anneal * KLD




In [ ]:
# 단순하게 torch.FloatTensor 쓰는 함수
def naive_sparse2tensor(data):
    return torch.FloatTensor(data.toarray())

# 위 함수를 변형한 함수 (속도 개선)
# 근데 조교님 정답에선 이거 안씀
# 어떻게 쓰는 건지 모르겠음
def sparse2torch_sparse(data):
    """
    scipy 행렬에서 torch 희소행렬로 L2 Norm을 이용하여 변환
    이렇게 하면 단순하게 torch.FloatTensor(data.toarray())를 쓰는 것 보다 빨라짐
    https://discuss.pytorch.org/t/sparse-tensor-use-cases/22047/2
    """
    samples = data.shape[0]
    features = data.shape[1]
    # coo행렬은 coordinate format을 이용하여 희소행렬을 표현하는 방법
    # 원소의 좌표와 data 를 함께 넘겨줌
    # 참고 https://radish-greens.tistory.com/1
    coo_data = data.tocoo()
    indices = torch.LongTensor([coo_data.row, coo_data.col])
    row_norms_inv = 1 / np.sqrt(data.sum(1))
    row2val = {i : row_norms_inv[i].item() for i in range(samples)}
    values = np.array([row2val[r] for r in coo_data.row])
    t = torch.sparse.FloatTensor(indices, torch.from_numpy(values).float(), [samples, features])
    return t


def train(model, criterion, optimizer, is_VAE = False):
    # Turn on training mode
    model.train()
    train_loss = 0.0
    start_time = time.time()
    global update_count
    # idxlist, N 는 아래 코드에서 따온 변수임
        # train_data = loader.load_data('train')
        # N = train_data.shape[0] : train data의 크기
        # idxlist = list(range(N)) : train data 크기만큼 range list
    
    # idxlist 를 랜덤으로 섞는다
    np.random.shuffle(idxlist)
    
    # 배치 단위로 잘라서 학습
    for batch_idx, start_idx in enumerate(range(0, N, args.batch_size)):
        # 끝부분 처리
        end_idx = min(start_idx + args.batch_size, N)
        # 랜덤으로 섞인 train_data 배치사이즈로 자르기
        data = train_data[idxlist[start_idx:end_idx]]
        # 텐서로 변환
        data = naive_sparse2tensor(data).to(device)
        optimizer.zero_grad()# 모든 매개변수의 변화도 버퍼를 0으로 만듦

        # Multi-VAE
        if is_VAE:
            #Default = 200000
            if args.total_anneal_steps > 0:
                anneal = min(args.anneal_cap, 
                                1. * update_count / args.total_anneal_steps)
            else:
                anneal = args.anneal_cap # default = 0.2

            # TODO
            # model에 입력 출력 코드를 작성해주세요
            recon_batch, mu, logvar = model(data)

            # loss 함수를 설정해주세요        
            # criterion 은 loss_function_vae 가 들어옴
            loss = criterion(recon_batch, data, mu, logvar, anneal)

        #Multi-DAE는 else로 처리
        else:
          recon_batch = model(data)
          loss = criterion(recon_batch, data)

        loss.backward()             # 역전파
        train_loss += loss.item()   # loss 값 적립
        optimizer.step()            # 업데이트 진행

        update_count += 1

        # 100 번(Log_interval) 마다 실행
        # 근데 배치가 51개로 설정되어 있어서 10으로 바꿔 쓰면 그제야 실행됨
        if batch_idx % args.log_interval == 0 and batch_idx > 0: # log_interval : default=100
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:4d}/{:4d} batches | ms/batch {:4.2f} | '
                    'loss {:4.2f}'.format(
                        epoch, 
                        batch_idx, len(range(0, N, args.batch_size)),
                        elapsed * 1000 / args.log_interval,
                        train_loss / args.log_interval))
            

            start_time = time.time()
            train_loss = 0.0


def evaluate(model, criterion, data_tr, data_te, is_VAE=False):
    # Turn on evaluation mode
    model.eval()
    total_loss = 0.0
    global update_count
    
    # 테스트 리스트
    e_idxlist = list(range(data_tr.shape[0]))
    e_N = data_tr.shape[0]
    n100_list = []
    r20_list = []
    r50_list = []
    
    # 학습 아닐때 no_grad 써주기
    with torch.no_grad():
        # 배치 크기로 자르기
        for start_idx in range(0, e_N, args.batch_size):
            # 배치 끝처리
            end_idx = min(start_idx + args.batch_size, N)
            # 테스트의 트레인 데이터 자르기
            data = data_tr[e_idxlist[start_idx:end_idx]]
            # 테스트의 테스트 데이터 불러오기
            heldout_data = data_te[e_idxlist[start_idx:end_idx]]

            data_tensor = naive_sparse2tensor(data).to(device)
            
            # Multi-VAE
            if is_VAE :
                if args.total_anneal_steps > 0:
                    anneal = min(args.anneal_cap, 
                                    1. * update_count / args.total_anneal_steps)
                else:
                    anneal = args.anneal_cap
                
                #TODO
                #model에 입력 출력 코드를 작성해주세요
                recon_batch, mu, logvar = model(data_tensor)
                #loss 함수를 설정해주세요
                loss = criterion(recon_batch, data_tensor, mu, logvar, anneal)

            # Multi-DAE
            else :
                recon_batch = model(data_tensor)
                loss = criterion(recon_batch, data_tensor)


            total_loss += loss.item()

            # Exclude examples from training set
            # 모델일 때는 cpu(), 텐서일 때는 cuda()로 쓴다고 함
            recon_batch = recon_batch.cpu().numpy()
            # data.nonzero() -> 배치단위로 잘린 데이터중 0이 아닌 값의 Index 값
            # 테스트의 트레인 데이터 값들을 마이너스 무한대로 변환
            recon_batch[data.nonzero()] = -np.inf

            # 평가 (METRIC)
            # 100개의 배치로 계산한 NDCG
            n100 = NDCG_binary_at_k_batch(recon_batch, heldout_data, 10)
            # 20개 배치로 계산한 Recall@K
            r20 = Recall_at_k_batch(recon_batch, heldout_data, 2)
            # 50개 배치로 계산한 Recall@K
            r50 = Recall_at_k_batch(recon_batch, heldout_data, 5)

            n100_list.append(n100)
            r20_list.append(r20)
            r50_list.append(r50)
    # loss 배치 단위로 평균내기
    total_loss /= len(range(0, e_N, args.batch_size))
    n100_list = np.concatenate(n100_list)
    r20_list = np.concatenate(r20_list)
    r50_list = np.concatenate(r50_list)

    return total_loss, np.mean(n100_list), np.mean(r20_list), np.mean(r50_list)


### Metric 정의

In [ ]:
import bottleneck as bn
import numpy as np

def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    '''
    Normalized Discounted Cumulative Gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    # 마이너스 무한대 값이 들어있는 X_pred, 테스트 데이터가 들어있는 heldout_batch
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)

    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]

    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG


def Recall_at_k_batch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

## MultiDAE 테스트

In [ ]:

###############################################################################
# Load data
###############################################################################

loader = DataLoader(args.data)

n_items = loader.load_n_items()
train_data = loader.load_data('train')
vad_data_tr, vad_data_te = loader.load_data('validation')
test_data_tr, test_data_te = loader.load_data('test')

N = train_data.shape[0]
idxlist = list(range(N))

###############################################################################
# Build the model
###############################################################################

p_dims = [200, 600, n_items]
model = MultiDAE(p_dims).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=args.wd)
criterion = loss_function_dae

###############################################################################
# Training code
###############################################################################

best_n100 = -np.inf
update_count = 0

In [ ]:
for epoch in range(1, args.epochs + 1):
    epoch_start_time = time.time()
    train(model, criterion, optimizer, is_VAE=False)
    val_loss, n100, r20, r50 = evaluate(model, criterion, vad_data_tr, vad_data_te, is_VAE=False)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:4.2f}s | valid loss {:4.2f} | '
            'n100 {:5.3f} | r20 {:5.3f} | r50 {:5.3f}'.format(
                epoch, time.time() - epoch_start_time, val_loss,
                n100, r20, r50))
    print('-' * 89)

    n_iter = epoch * len(range(0, N, args.batch_size))


    # Save the model if the n100 is the best we've seen so far.
    if n100 > best_n100:
        with open("MultiDAE.pt", 'wb') as f:
            torch.save(model, f)
        best_n100 = n100



# # Load the best saved model.
# with open(args.save, 'rb') as f:
#     model = torch.load(f)

# # Run on test data.
# test_loss, n100, r20, r50 = evaluate(model, criterion, test_data_tr, test_data_te, is_VAE=False)
# print('=' * 89)
# print('| End of training | test loss {:4.2f} | n100 {:4.2f} | r20 {:4.2f} | '
#         'r50 {:4.2f}'.format(test_loss, n100, r20, r50))
# print('=' * 89)

In [ ]:
with open("MultiDAE.pt", 'wb') as f:
    torch.save(model, f)

In [ ]:
def predict(model, data_tr, is_VAE=False):
    model.eval()
    global update_count
    users = []
    items = []
    with torch.no_grad():
        for start_idx in range(data_tr.shape[0]):
            data = data_tr[start_idx]
            data_tensor = naive_sparse2tensor(data).to(device)            
            # Multi-VAE
            if is_VAE :        
                recon_batch, mu, logvar = model(data_tensor)
            # Multi-DAE
            else:
                recon_batch = model(data_tensor)
            recon_batch = recon_batch.cpu().numpy()
            recon_batch[data.nonzero()] = -np.inf
            
            for rec in recon_batch:
                up = np.argpartition(rec, -10)[-10:].tolist()
                users.extend([start_idx] * 10)
                items.extend(up)
    user2rec_list = pd.DataFrame({'user': users, 'item': items}, dtype=int)
    return user2rec_list

In [ ]:
sub_data = loader.load_data('sub')

In [ ]:
with open("MultiDAE.pt", 'rb') as f:
    model = torch.load(f)    
sub_data = loader.load_data('sub')
user2rec_list = predict(model, sub_data, is_VAE=False)
user2rec_list['user'] = user2rec_list['user'].map(id2user)
user2rec_list['item'] = user2rec_list['item'].map(id2item)
Multi_DAE = user2rec_list.sort_values(by=['user','item'])
Multi_DAE

## MultiVAE 테스트 (TODO)

- 위의 MultiVAE 모델 코드, train, evaluate 함수를 완성하여, 아래 훈련 코드가 정상적으로 동작하도록 해보세요!
- 다양한 VAE의 코드를 다음 코드를 확인한 뒤에, 아래코드에 맞춰서 직접 작성해보는 연습을 해보세요!
  - https://github.com/AntixK/PyTorch-VAE
- 완성해야할 함수
  - MultiVAE class
    - forward
    - encode
    - decode
  - train
  - evaluate

In [ ]:

###############################################################################
# Load data
###############################################################################

loader = DataLoader(args.data)

n_items = loader.load_n_items()
train_data = loader.load_data('train')
vad_data_tr, vad_data_te = loader.load_data('validation')
test_data_tr, test_data_te = loader.load_data('test')

N = train_data.shape[0]
idxlist = list(range(N))

###############################################################################
# Build the model
###############################################################################

p_dims = [200, 600, n_items]
model = MultiVAE(p_dims).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=args.wd)
criterion = loss_function_vae

###############################################################################
# Training code
###############################################################################

best_n100 = -np.inf
update_count = 0

In [ ]:
for epoch in range(1, args.epochs + 1):
    epoch_start_time = time.time()
    train(model, criterion, optimizer, is_VAE=True)
    val_loss, n100, r20, r50 = evaluate(model, criterion, vad_data_tr, vad_data_te, is_VAE=True)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:4.2f}s | valid loss {:4.2f} | '
            'n100 {:5.3f} | r20 {:5.3f} | r50 {:5.3f}'.format(
                epoch, time.time() - epoch_start_time, val_loss,
                n100, r20, r50))
    print('-' * 89)

    n_iter = epoch * len(range(0, N, args.batch_size))


    # Save the model if the n100 is the best we've seen so far.
    if n100 > best_n100:
        with open("MultiVAE.pt", 'wb') as f:
            torch.save(model, f)
        best_n100 = n100



# # Load the best saved model.
# with open("MultiVAE.pt", 'rb') as f:
#     model = torch.load(f)

# # Run on test data.
# test_loss, n100, r20, r50 = evaluate(model, criterion, test_data_tr, test_data_te, is_VAE=True)
# print('=' * 89)
# print('| End of training | test loss {:4.2f} | n100 {:4.2f} | r20 {:4.2f} | '
#         'r50 {:4.2f}'.format(test_loss, n100, r20, r50))
# print('=' * 89)

In [ ]:
with open("MultiVAE.pt", 'wb') as f:
    torch.save(model, f)

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.



In [ ]:
with open("MultiVAE.pt", 'rb') as f:
    model = torch.load(f)
sub_data = loader.load_data('sub')


In [ ]:
user2rec_list = predict(model, sub_data[22], is_VAE=True)


In [ ]:
user2rec_list['user'] = user2rec_list['user'].map(id2user)


In [ ]:
user2rec_list['item'] = user2rec_list['item'].map(id2item)

In [ ]:
pd.DataFrame(list(id2user.items()), columns=['1', '2']).to_csv("id2user.csv", index = False)

In [ ]:
pd.DataFrame(list(id2item.items()), columns=['1', '2']).to_csv("id2item.csv", index = False)